Author: Guy, Michael, Joshua
Summary: This is a comprenhensive simulation engine for ambulance simulation. It uses the solution from two-stage stochastic program and robust programs to simulate how ambulances would perform. The simulation mostly runs using a priority-queue of events with respect to time. Events include things like ambulance arrives to the scene of call, ambulance arrives to hospital, ect.

In [14]:
import Pkg #this line doesn't seem to run either way for me
Pkg.add("Pkg")
Pkg.add("Dates")

  Resolving package versions...
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Manifest.toml`
  Resolving package versions...
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Manifest.toml`


In [15]:
"""This activates packages, namely ambulance deployment"""

guypath = "/Users/guyfarmer/ambulance_connect/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/"
michaelpath= "/home/mhilborn/Desktop/UT-Spring2021/SeniorLab/AmbulanceDeployement/AmbulanceDeployment.jl-legacy"
joshpath = "C:/Users/Owner/Documents/Austin/AmbulanceDeployment/AmbulanceDeployment.jl-legacy/"

currentpath = joshpath

cd(currentpath)
Pkg.activate(".")

 Activating environment at `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`


Warning: Package AmbulanceDeployment does not have Random in its dependencies:
│ - If you have AmbulanceDeployment checked out for development and have
│   added Random as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with AmbulanceDeployment
└ Loading Random into AmbulanceDeployment from project dependency, future warnings for AmbulanceDeployment are suppressed.

In [16]:
using AmbulanceDeployment

In [17]:
using DataFrames, JLD, Distributions, CSV, Random, JSON, DelimitedFiles, Dates #Plots I don't think we use Plots
import DataStructures: PriorityQueue, enqueue!, dequeue!
include(".//src//problem.jl")
include(".//src//model.jl")

In [18]:
"""Helper functions for the simuation engine?"""
#=
Author : Ng Yeesian
Modified : Guy Farmer / Zander Tedjo / Will Worthington / Michael Hilborn
generates the closest dispatch model
Summary :
Input :
Output :
=#
mutable struct ClosestDispatch <: DispatchModel
    drivetime::DataFrame
    candidates::Vector{Vector{Int}}
    available::Vector{Int}
    assignment::Vector{Int} # which location the ambulance is assigned to
    ambulances::Vector{Vector{Int}} # list of ambulances assigned to each location
    status::Vector{Symbol} # the current status of the ambulance
        # possible statuses: :available, :responding, :atscene, :conveying, :returning
    fromtime::Vector{Int} # the time it started the new status
    hospital::Vector{Int}
end

#=
Summary :
Input :
Output : ClosestDispatch struct
=#
function ClosestDispatch(p::DeploymentProblem, drivetime::DataFrame,distribution::Vector{Int})
    candidates = Any[]
    I = 1:p.nlocations
    for region in 1:p.nregions
        push!(candidates, I[vec(p.coverage[region,:])])
    end
    #println("creating the dispatch problem :$distribution")

    assignment = zeros(Int, p.nambulances)
    ambulances = [Int[] for i in 1:p.nlocations]
    k = 1
    for i in eachindex(distribution), j in 1:distribution[i]
        assignment[k] = i
        push!(ambulances[i], k)
        k += 1
    end
    status = fill(:available, p.nambulances)
    fromtime = zeros(Int, p.nambulances)
    hospital = zeros(Int, p.nambulances)
    
    ClosestDispatch(drivetime, candidates, distribution, assignment, ambulances, status, fromtime, hospital)

end

function ClosestDispatch(p::DeploymentProblem, problem::DispatchProblem)
    stn_names = [Symbol("stn$(i)_min") for i in 1:size(p.coverage,2)]
    ClosestDispatch(p, problem.emergency_calls[:, stn_names], problem.available)
end

function update_ambulances!(model::ClosestDispatch, i::Int, delta::Int)
    model.available[i] += delta
    @assert model.available[i] >= 0
end

function closest_available(dispatch::ClosestDispatch, id::Int, problem::DispatchProblem)
    location = 0
    min_time = Inf
    for i in dispatch.candidates[problem.emergency_calls[id, :neighborhood]]
        if problem.available[i] > 0 && dispatch.drivetime[id, i] < min_time
            #println("apparently this station is available : $i and here is the amount of ambulances avalable: $(problem.available[i])")
            location = i
            min_time = dispatch.drivetime[id, i]
        end
    end
    location
end

function respond_to!(dispatch::ClosestDispatch, i::Int, t::Int)
    @assert length(dispatch.ambulances[i]) > 0 "$i: $(dispatch.ambulances[i])"
    amb = popfirst!(dispatch.ambulances[i])
    # @assert redeploy.hospital[amb] == 0
    @assert amb != 0
    @assert dispatch.status[amb] == :available "$amb: $(dispatch.status[amb])"
    dispatch.status[amb] = :responding
    #dispatch.fromtime[amb] = t
    amb
end

function arriveatscene!(dispatch::ClosestDispatch, amb::Int, t::Int)
    @assert dispatch.status[amb] == :responding "$amb: $(dispatch.status[amb])"
    @assert dispatch.hospital[amb] == 0
    dispatch.status[amb] = :atscene
    dispatch.fromtime[amb] = t
end

function arriveathospital!(dispatch::DispatchModel, amb::Int, hosp::Int, t::Int)
    @assert dispatch.status[amb] == :atscene "$amb: $(dispatch.status[amb])"
    @assert dispatch.hospital[amb] != 0
    dispatch.status[amb] = :atHospital
    dispatch.fromtime[amb] = t
    dispatch.hospital[amb] = hosp
end

function returning_to!(dispatch::DispatchModel, amb::Int, t::Int)
    @assert dispatch.status[amb] == :atHospital "$amb: $(dispatch.status[amb])"
    @assert dispatch.hospital[amb] != 0
    dispatch.status[amb] = :returning
    dispatch.fromtime[amb] = t
    dispatch.assignment[amb]
end

returning_to! (generic function with 1 method)

In [50]:
""" simulation parameters """
turnaround = Distributions.LogNormal(3.65, 0.3)#Distributions.LogNormal(3.089, 0.3) #Distributions.LogNormal(3.65, 0.3)
lambda = 0
local_path = ""
model_name = "Stochastic"
namb = 40
ncalls = 1000
#using AmbulanceDeployment

pwd()

cd(currentpath * "/test/")
pwd()

num_regions = 3199
hospitals = CSV.File(string(local_path,"../test/austin_data_3200/hospitals.csv")) |> DataFrame
stations = CSV.File(string(local_path,"../test/austin_data_3200/stations.csv")) |> DataFrame
hourly_calls = CSV.File("../test/austin_data_3200/WeekdayCalls.csv") |> DataFrame
adjacent_nbhd = CSV.File("../test/austin_data_3200/adjacent_nbhd.csv") |> DataFrame
test_calls = CSV.File("../test/austin_data_3200/austin_test_calls_v3.csv")|> DataFrame 
coverage = CSV.read("../test/austin_data_3200/coverage_regression.csv", DataFrame, header=false) #coverage_real is austin-data
coverage = delete!(coverage,1) #trim df header (1st row)
coverage = select(coverage,2:3234) #trim df header (1st column)
coverage = Matrix(coverage)
#coverage = convert(Array{Bool, 2}, coverage)
coverage = convert(Array{Bool,2},coverage')

# num_regions = 200
# hospitals = CSV.File(string(local_path,"../test/austin_data/hospitals.csv")) |> DataFrame
# stations = CSV.File(string(local_path,"../test/austin_data/stations.csv")) |> DataFrame
# hourly_calls = CSV.File("../test/austin_data/WeekdayCalls.csv") |> DataFrame
# adjacent_nbhd = CSV.File("../test/austin_data/adjacent_nbhd.csv") |> DataFrame
#CALLS IN MINUTES #test_calls = CSV.File("../test/austin_data/austin_test_calls_v1.csv")|> DataFrame  
# coverage = CSV.read("../test/austin_data/coverage.csv", DataFrame, header=false) #coverage_real is austin-data
# coverage = Matrix(coverage)
# coverage = convert(Array{Bool, 2}, coverage)

solverstats = JLD.load("../src/outputs/austin_team_stats_1_03.jld")
amb_deployment = solverstats["amb_deployment"]
model_dict = Dict{String, Symbol}("Stochastic"=>:Stochastic, "Robust01"=>:Robust01, "Robust005"=>:Robust005, "Robust001"=>:Robust001, "Robust0001"=>:Robust0001,
          "Robust00001"=>:Robust00001, "MEXCLP"=>:MEXCLP, "MALP"=>:MALP)


include("..//src//problem.jl")

p = DeploymentProblem(
          num_regions,
          hourly_calls,
          adjacent_nbhd,
          coverage,
          namb = namb,
          train_filter = (hourly_calls[!,:year] .== 2019) .* (hourly_calls[!,:month] .<= 3)
          )

print("n ambulances ",p.nambulances)
print("\n locations  ",p.nlocations)
print("\n nregions ",p.nregions)
# print("\n train ", p.train)
# print("\n test ", p.test)
print("\n coverage ",size(p.coverage))
print("\n demand ",size(p.demand))
print("\n adjacency ",size(p.adjacency))

#update 1: remove negative indices
test_calls = test_calls[1:ncalls,:]
println("running $namb ambuances & $ncalls calls")

# distribution of ambulances across all of the stations
x = amb_deployment[model_dict[model_name]][namb]#convert(Array{Int,1},ones(44))#amb_deployment[model_dict[model_name]][namb]

#establishes the dispatch problem
problem = DispatchProblem(test_calls, hospitals, stations, p.coverage, x, turnaround=turnaround)

#establishes the dispatch model
dispatch = ClosestDispatch(p, problem)
Random.seed!(1234)

40
44
3199
(0, 2)
(0,)
(0,)
(3233, 44)
Array{Bool,2}
(3199, 3199)
BitArray{2}
n ambulances 40
 locations  44
 nregions 3199
 coverage (3233, 44)
 demand (0, 2)
 adjacency (3199, 3199)running 40 ambuances & 1000 calls


MersenneTwister(UInt32[0x000004d2], Random.DSFMT.DSFMT_state(Int32[-1393240018, 1073611148, 45497681, 1072875908, 436273599, 1073674613, -2043716458, 1073445557, -254908435, 1072827086  …  -599655111, 1073144102, 367655457, 1072985259, -1278750689, 1018350124, -597141475, 249849711, 382, 0]), [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], UInt128[0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000  …  0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x000000000000

###### problem.available

In [51]:
#test_calls
#problem.emergency_calls[:, Symbol("stn$(1)_min")]

In [52]:
function logRegression(gridTime)
    if(gridTime < 0)
        print("grid time less than 0: ", gridTime)
    end
    gridTime = gridTime / 60
    slope = 0.23868463365149437
    intercept = 1.261777659720721
    ax = log(gridTime)*slope
    new_time = exp(ax + intercept)
    new_time =  new_time * 60
    new_time
end

logRegression (generic function with 1 method)

In [58]:
"""Simulation Engine"""

Pkg.add("Query")
using Query

struct EMSEngine{T}
    eventlog::DataFrame
    eventqueue::PriorityQueue{T,Int,Base.Order.ForwardOrdering}
    shortqueue::PriorityQueue{T,Int,Base.Order.ForwardOrdering}
    guiArray::Array{Any, 1}
end

struct gui_event
    #call made / call responded / ambulance arrived / shortfall
    event_type::String
    #neighborhood where call is made from
    neighborhood_id::Int
    # station where call is made from
    deployment_id::Int
    # id of the event generated
    event_id::Int
    ambulance_id::Int
    arrival_time::Int
    remaining_amb::Int
    # current timestamp of event
    timestamp::DateTime
end

## generates the priority queue and instantiates the EMSEngine struct
function generateCalls(problem::DispatchProblem)
    ncalls = nrow(problem.emergency_calls)
    eventlog = DataFrame(
        id = 1:ncalls,
        dispatch_from = zeros(Int, ncalls),
        waittime = fill(0.0, ncalls),
        responsetime = fill(Inf, ncalls),
        scenetime = fill(Inf, ncalls),
        conveytime = fill(Inf, ncalls),
        returntime = fill(Inf, ncalls),
        return_to = zeros(Int, ncalls),
        return_type = fill(:station, ncalls),
        hospital = zeros(Int, ncalls),
        ambulance = zeros(Int, ncalls)
    )
    eventqueue = PriorityQueue{Tuple{Symbol,Int,Int,Int},Int}()
    shortqueue = PriorityQueue{Tuple{Symbol,Int,Int,Int},Int}()
    guiArray = Any[]
    for i in 1:nrow(problem.emergency_calls)
        t = problem.emergency_calls[i, :arrival_seconds]
        enqueue!(eventqueue, (:call, i, t, problem.emergency_calls[i, :neighborhood]), t)
    end
    EMSEngine{Tuple{Symbol,Int,Int,Int}}(eventlog, eventqueue, shortqueue, guiArray) ## returns the ems engine struct
end

function simulate_events!(problem::DispatchProblem, dispatch::ClosestDispatch)
    engine = generateCalls(problem)
    while(~isempty(engine.eventqueue))
        (event, id, t, value) = dequeue!(engine.eventqueue)

        if event == :call
            # if the event is reachable then change event to station2call
            # if not it is a shortfall
            println("calling event id: $id time: $t value: $value")
            call_event!(engine, problem, dispatch, id, t, value)
           
        elseif event == :station2call
            #once the ambulance arrives at the station change event to call2hospital
            println("Ambulance has arrived on the scene for event $id at time $t")
            station2call_event!(engine, problem, dispatch, id, t, value)
        elseif event == :call2hospital
            #once the ambulance arrives at the hospital it will hand over the patient and then 
            #change the deploy back to its station
            println("Ambulance for event $id has arrived at the hospital at time $t")
            call2hosp_event!(engine, problem, dispatch, id, t, value)

        elseif event == :hospital2station
            #once the ambulance drops off the patient it will reroute to the station
            println("Ambulance for event $id is returning to the station at $t")
            return_event!(engine, problem, dispatch, id, t, value)
            
        elseif event == :done
            #increase the number of ambulances at that station now that it is done
            done_event!(engine, problem, dispatch, id, t, value)
            
        end
    end
    println("This is the Short Queue")
    while(~isempty(engine.shortqueue))
       (event, id, t, value) = dequeue!(engine.shortqueue)
        print("$event, $id, $t, $value")
        print("This is not empty")
    end
    engine.eventlog , engine.guiArray
end


function call_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int, # the id of the emergency call
        t::Int, # the time of the emergency call
        nbhd::Int; # the neighborhood the call is from
    )

    #check if there is an ambulance within the coverage matrix
    if sum(problem.deployment[problem.coverage[nbhd,:]]) == 0
        @assert false "$id: no ambulance reachable for call at $nbhd"
    #check if one of the ambulances within the coverage is available
    elseif sum(problem.available[problem.coverage[nbhd,:]]) > 0
        i = closest_available(dispatch, id, problem)
        println("closest available station : $i")
        @assert i > 0 "$id: dispatch from $i to nbhd $nbhd" # assume valid i (enforced by <if> condition)
        update_ambulances!(dispatch, i, -1)
        ems.eventlog[id, :dispatch_from] = i
        @assert problem.available[i] >= 0
        
        travel_time = ceil(Int, problem.emergency_calls[id, Symbol("stn$(i)_min")]) #60*
        println(travel_time)
        #travel_time = logRegression(travel_time) # gridTime2Bin(travel_time) #adjust time
        travel_time = ceil(travel_time) #time mod
        println(travel_time)
        @assert travel_time >= 0
        if(t != problem.emergency_calls[id, :arrival_seconds])
            print("This is a shortfall")
        end
        ems.eventlog[id, :responsetime] = (travel_time + t - problem.emergency_calls[id,:arrival_seconds]) / 60 # minutes

        amb = respond_to!(dispatch, i, t)
        ems.eventlog[id, :ambulance] = amb

        event = gui_event("call responded", problem.emergency_calls[id, :neighborhood], i, id, amb,-1, problem.available[i] ,Dates.now())
        num_ambulances_array = Array{Integer}(undef, size(problem.available, 1))
        for i in 1:size(problem.available, 1)
            num_ambulances_array[i] = problem.available[i]
        end
        push!(ems.guiArray,event)
        push!(ems.guiArray,num_ambulances_array)
        enqueue!(ems.eventqueue, (:station2call, id, t + travel_time, amb), t + travel_time)
    #else queue it
    else
        # println(id, ": call from ", nbhd, " queued behind ", problem.wait_queue[nbhd])
        # event = gui_event("call made", nbhd, -1, id, -1,-1,-1,Dates.now())
        # push!(ems.guiArray,event)
        # problem.shortfalls = problem.shortfalls + 1
        # #push!(problem.shortfalls) # count shortfalls
        # push!(problem.wait_queue[nbhd], id) # queue the emergency call
        event = gui_event("shortfall", problem.emergency_calls[id, :neighborhood], -1, id, -1,-1,-1 ,Dates.now())
        push!(ems.guiArray, event)
        enqueue!(ems.shortqueue, (:call, id, t, problem.emergency_calls[id, :neighborhood]), t)
        println(id, ": call from ", nbhd, " is a shortfall")
        println("and there are this many amb available: ", sum(problem.available))
    end
end

function station2call_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int, # the id of the emergency call
        t::Int, # the time of the emergency call
        amb::Int
    )
    arriveatscene!(dispatch, amb, t)
    # time the ambulance spends at the scene
    scene_time = ceil(Int,60*0.4*rand(problem.turnaround)) # 60sec*0.4*mean(40minutes) ~ 15minutes
    ems.eventlog[id, :scenetime] = scene_time / 60 # minutes
    @assert scene_time > 0
    event = gui_event("ambulance arrived", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    enqueue!(ems.eventqueue, (:call2hospital, id, t + scene_time, amb), t + scene_time)
end

function call2hosp_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int, # the id of the emergency call
        t::Int, # the time of the emergency call
        amb::Int
    )
    h = let mintime = Inf, minindex = 0
        for h in 1:nrow(problem.hospitals)
            traveltime = problem.emergency_calls[id, Symbol("hosp$(h)_min")]
            if !isna(traveltime) && !(traveltime=="NA") && traveltime < mintime
                @assert traveltime >= 0
                minindex = h; mintime = traveltime
            end
        end
        minindex
    end
    @assert h != 0
    dispatch.hospital[amb] = ems.eventlog[id, :hospital] = h
    #60*15 + ceil(Int, 60*problem.emergency_calls[id, Symbol("hosp$(h)_min")]) # ~20minutes why the hell would we add 15 minutes and then also the hopsital min!! smh
    conveytime = ceil(Int, 60*problem.emergency_calls[id, Symbol("hosp$(h)_min")]) # ~20minutes
    conveytime = ceil(logRegression(conveytime)) #adjust time
    ems.eventlog[id, :conveytime] = conveytime / 60 # minutes
    @assert conveytime >= 0 conveytime
    arriveathospital!(dispatch, amb, h, t)
    
    #         #call made / call responded / ambulance arrived / shortfall
#     event_type::String
#     #neighborhood where call is made from
#     neighborhood_id::Int
#     # station where call is made from
#     deployment_id::Int
#     # id of the event generated
#     event_id::Int
#     ambulance_id::Int
#     arrival_time::Int
#     remaining_amb::Int
#     # current timestamp of event
#     timestamp::DateTime
    event = gui_event("ambulance hospital", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    
    
    enqueue!(ems.eventqueue, (:hospital2station, id, t+conveytime, amb), t+conveytime)
end

function return_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int,
        t::Int,
        amb::Int
    )
    stn = returning_to!(dispatch, amb, t)
    h = dispatch.hospital[amb]
    returntime = ceil(Int,60*problem.hospitals[h, Symbol("stn$(stn)_min")]) # ~ 10minutes
    #returntime = ceil(returntime * .9) #time mod
    dispatch.hospital[amb] = 0
    ems.eventlog[id, :returntime] = returntime / 60 # minutes
    @assert returntime >= 0 returntime
    t_end = t + returntime
    
#         #call made / call responded / ambulance arrived / shortfall
#     event_type::String
#     #neighborhood where call is made from
#     neighborhood_id::Int
#     # station where call is made from
#     deployment_id::Int
#     # id of the event generated
#     event_id::Int
#     ambulance_id::Int
#     arrival_time::Int
#     remaining_amb::Int
#     # current timestamp of event
#     timestamp::DateTime
    
    event = gui_event("ambulance returning", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    
    
    enqueue!(ems.eventqueue, (:done, id, t_end, amb), t_end)
    # num_ambulances_array = Array{Integer}(undef, size(problem.available, 1))
    # for i in 1:size(problem.available, 1)
    #     num_ambulances_array[i] = problem.available[i]
    # end
    # push!(ems.guiArray,num_ambulances_array)
end

function done_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int,
        t::Int,
        amb::Int
    )
    i = dispatch.assignment[amb]
    println("Ambulance from event $id has returned to station $i")
    @assert i > 0 "$id: dispatch from $i to nbhd $nbhd" # assume valid i (enforced by <if> condition)
    update_ambulances!(dispatch, i, 1)
    push!(dispatch.ambulances[i], amb)
    dispatch.status[amb] = :available
    ems.eventlog[id, :return_to] = id
    @assert problem.available[i] > 0
    
#         #call made / call responded / ambulance arrived / shortfall
#     event_type::String
#     #neighborhood where call is made from
#     neighborhood_id::Int
#     # station where call is made from
#     deployment_id::Int
#     # id of the event generated
#     event_id::Int
#     ambulance_id::Int
#     arrival_time::Int
#     remaining_amb::Int
#     # current timestamp of event
#     timestamp::DateTime
    
    event = gui_event("ambulance returned", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    
    while(~isempty(ems.shortqueue))
        # record time difference
        (event, short_id, short_t, value) = dequeue!(ems.shortqueue)
        enqueue!(ems.eventqueue, (event, short_id, t , value), short_t)
    end
end


  Resolving package versions...
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Manifest.toml`


done_event! (generic function with 1 method)

In [59]:
"""call simulation"""
#@time df, guiArray = simulate_events!(problem, dispatch)
df, guiArray = simulate_events!(problem, dispatch)

calling event id: 1 time: 38 value: 1012
closest available station : 1
143
143
Ambulance has arrived on the scene for event 1 at time 181
calling event id: 2 time: 575 value: 220
closest available station : 31
646
646
calling event id: 3 time: 673 value: 1687
closest available station : 25
477
477
calling event id: 4 time: 691 value: 1536
closest available station : 33
328
328
Ambulance for event 1 has arrived at the hospital at time 864
Ambulance has arrived on the scene for event 4 at time 1019
Ambulance has arrived on the scene for event 3 at time 1150
calling event id: 5 time: 1166 value: 1621
closest available station : 14
375
375
calling event id: 6 time: 1187 value: 578
closest available station : 22
222
222
Ambulance has arrived on the scene for event 2 at time 1221
Ambulance has arrived on the scene for event 6 at time 1409
calling event id: 7 time: 1450 value: 1169
closest available station : 16
927
927
Ambulance has arrived on the scene for event 5 at time 1541
calling event

Ambulance from event 24 has returned to station 17
calling event id: 32 time: 8673 value: 1008
closest available station : 8
272
272
Ambulance for event 23 is returning to the station at 8723
Ambulance has arrived on the scene for event 31 at time 8819
Ambulance from event 22 has returned to station 9
Ambulance has arrived on the scene for event 32 at time 8945
Ambulance for event 26 has arrived at the hospital at time 8946
Ambulance for event 25 has arrived at the hospital at time 8947
Ambulance from event 23 has returned to station 15
calling event id: 33 time: 9154 value: 575
closest available station : 2
342
342
Ambulance for event 27 has arrived at the hospital at time 9208
Ambulance for event 30 has arrived at the hospital at time 9290
calling event id: 34 time: 9489 value: 1501
closest available station : 14
619
619
Ambulance has arrived on the scene for event 33 at time 9496
Ambulance for event 31 has arrived at the hospital at time 9564
calling event id: 35 time: 9626 value: 8

Ambulance from event 44 has returned to station 30
Ambulance for event 51 has arrived at the hospital at time 16337
Ambulance for event 53 has arrived at the hospital at time 16456
Ambulance has arrived on the scene for event 57 at time 16477
Ambulance from event 48 has returned to station 1
Ambulance from event 47 has returned to station 43
Ambulance for event 56 has arrived at the hospital at time 16936
Ambulance for event 52 is returning to the station at 16966
Ambulance for event 49 is returning to the station at 17076
Ambulance for event 54 has arrived at the hospital at time 17081
Ambulance for event 50 is returning to the station at 17137
calling event id: 58 time: 17141 value: 1316
closest available station : 1
541
541
Ambulance for event 53 is returning to the station at 17232
calling event id: 59 time: 17297 value: 1438
closest available station : 17
715
715
Ambulance from event 52 has returned to station 5
Ambulance for event 51 is returning to the station at 17343
Ambulance

closest available station : 4
254
254
Ambulance has arrived on the scene for event 83 at time 27032
Ambulance for event 76 is returning to the station at 27073
Ambulance for event 79 has arrived at the hospital at time 27137
Ambulance has arrived on the scene for event 82 at time 27210
Ambulance for event 77 is returning to the station at 27215
Ambulance has arrived on the scene for event 81 at time 27377
Ambulance from event 73 has returned to station 28
calling event id: 84 time: 27471 value: 513
closest available station : 2
93
93
Ambulance for event 78 is returning to the station at 27549
Ambulance from event 76 has returned to station 14
Ambulance has arrived on the scene for event 84 at time 27564
Ambulance has arrived on the scene for event 80 at time 27655
calling event id: 85 time: 27753 value: 1010
closest available station : 5
333
333
Ambulance for event 83 has arrived at the hospital at time 27779
Ambulance for event 82 has arrived at the hospital at time 27911
calling even

closest available station : 17
407
407
calling event id: 108 time: 38739 value: 1624
closest available station : 4
680
680
Ambulance has arrived on the scene for event 106 at time 38767
Ambulance from event 102 has returned to station 9
Ambulance has arrived on the scene for event 107 at time 39049
Ambulance from event 101 has returned to station 22
calling event id: 109 time: 39060 value: 1808
closest available station : 14
748
748
Ambulance for event 105 has arrived at the hospital at time 39182
Ambulance for event 103 is returning to the station at 39304
Ambulance for event 104 has arrived at the hospital at time 39409
Ambulance has arrived on the scene for event 108 at time 39419
Ambulance for event 106 has arrived at the hospital at time 39657
Ambulance from event 103 has returned to station 1
Ambulance has arrived on the scene for event 109 at time 39808
Ambulance for event 107 has arrived at the hospital at time 39890
Ambulance for event 105 is returning to the station at 40115


...

closest available station : 17
235
235
Ambulance for event 912 is returning to the station at 257742
Ambulance from event 910 has returned to station 15
calling event id: 919 time: 257797 value: 712
closest available station : 43
441
441
Ambulance has arrived on the scene for event 917 at time 257818
Ambulance has arrived on the scene for event 918 at time 257826
Ambulance for event 911 is returning to the station at 258006
calling event id: 920 time: 258007 value: 1135
closest available station : 4
528
528
Ambulance from event 909 has returned to station 30
calling event id: 921 time: 258031 value: 1984
closest available station : 26
369
369
Ambulance for event 916 has arrived at the hospital at time 258104
Ambulance for event 914 is returning to the station at 258116
calling event id: 922 time: 258191 value: 1985
closest available station : 17
272
272
Ambulance has arrived on the scene for event 919 at time 258238
calling event id: 923 time: 258327 value: 762
closest available statio

Ambulance has arrived on the scene for event 943 at time 262748
Ambulance has arrived on the scene for event 947 at time 262761
Ambulance from event 929 has returned to station 26
Ambulance for event 941 has arrived at the hospital at time 262814
Ambulance for event 931 is returning to the station at 262855
calling event id: 948 time: 262908 value: 581
closest available station : 22
628
628
calling event id: 949 time: 262939 value: 1868
closest available station : 4
884
884
Ambulance for event 934 is returning to the station at 262986
Ambulance for event 936 is returning to the station at 263047
Ambulance for event 940 has arrived at the hospital at time 263096
Ambulance from event 928 has returned to station 44
Ambulance for event 935 is returning to the station at 263134
Ambulance for event 946 has arrived at the hospital at time 263146
Ambulance from event 937 has returned to station 3
calling event id: 950 time: 263190 value: 1194
closest available station : 3
467
467
Ambulance for

closest available station : 26
477
477
Ambulance from event 961 has returned to station 11
Ambulance from event 957 has returned to station 19
Ambulance has arrived on the scene for event 967 at time 268597
Ambulance from event 960 has returned to station 8
Ambulance from event 958 has returned to station 26
Ambulance has arrived on the scene for event 968 at time 268754
Ambulance for event 964 is returning to the station at 268757
calling event id: 970 time: 268763 value: 1802
closest available station : 17
224
224
Ambulance for event 965 is returning to the station at 268773
Ambulance for event 966 has arrived at the hospital at time 268802
Ambulance has arrived on the scene for event 969 at time 268804
Ambulance from event 963 has returned to station 41
Ambulance has arrived on the scene for event 970 at time 268987
Ambulance from event 965 has returned to station 22
Ambulance from event 962 has returned to station 44
Ambulance for event 969 has arrived at the hospital at time 26957

236
236
Ambulance has arrived on the scene for event 993 at time 280670
Ambulance for event 991 has arrived at the hospital at time 280803
Ambulance for event 989 is returning to the station at 280843
calling event id: 994 time: 280862 value: 1261
closest available station : 4
388
388
Ambulance for event 990 is returning to the station at 281117
Ambulance from event 989 has returned to station 17
Ambulance from event 987 has returned to station 44
Ambulance for event 992 has arrived at the hospital at time 281232
Ambulance has arrived on the scene for event 994 at time 281250
Ambulance for event 993 has arrived at the hospital at time 281398
Ambulance from event 990 has returned to station 17
calling event id: 995 time: 281510 value: 1574
closest available station : 30
331
331
Ambulance for event 991 is returning to the station at 281758
Ambulance has arrived on the scene for event 995 at time 281841
Ambulance from event 991 has returned to station 17
Ambulance for event 992 is returni

(1000×11 DataFrame
  Row │ id     dispatch_from  waittime  responsetime  scenetime  conveytime  r ⋯
      │ Int64  Int64          Float64   Float64       Float64    Float64     F ⋯
──────┼─────────────────────────────────────────────────────────────────────────
    1 │     1              1       0.0       2.38333   11.3833      15.3167    ⋯
    2 │     2             31       0.0      10.7667    13.0         18.7167
    3 │     3             25       0.0       7.95      12.2667      17.4333
    4 │     4             33       0.0       5.46667   13.35        18.3833
    5 │     5             14       0.0       6.25      11.25        15.9167    ⋯
    6 │     6             22       0.0       3.7       16.25        16.2833
    7 │     7             16       0.0      15.45      15.6167      18.5333
    8 │     8              8       0.0       2.55      19.8333      12.4833
    9 │     9             22       0.0       6.45       9.08333     15.7667    ⋯
   10 │    10              8       0.0 

In [60]:
println(mean(df[!,"responsetime"]))
println(mean(df[!,"scenetime"]))
println(mean(df[!,"conveytime"]))
println(mean(df[!,"returntime"]))

8.145316666666668
16.25061666666667
15.4032
9.455366666666668


## Temp results journal 

println(mean(df["responsetime"]))
println(mean(df["scenetime"]))
println(mean(df["conveytime"]))
println(mean(df["returntime"]))
println(mean(df[!,"returntime"]))
#### lognromal 3.08
9.813166666666667 <br>
9.134116666666667 <br>
24.4176 <br>
8.908 <br>
#### lognromal 3.65
10.3443 <br>
16.000433333333337 <br>
24.4176 <br>
9.090300000000001 <br>
#### adjust travel time by time_bin weights
10.466350000000002 <br>
16.000433333333334 <br>
24.4176 <br>
9.073 <br>
#### global
7.7477 <br>
16.250616666666666 <br>
24.4176 <br>
9.114916666666668 <br>
#### log-log 3200 grid UNIFORM 44 WHOOPS
5.3978 <br>
16.000433333333337 <br>
15.4032 <br>
8.835816666666668 <br>
#### 3200 grid UNIFORM 44  WHOOPS
6.387766666666668 <br>
16.000433333333334 <br>
15.4032 <br>
8.805600000000002 <br>
#### log-log 3200 grid stochastic
5.6897166666666665 <br>
16.000433333333337 <br>
15.4032 <br>
9.427483333333335 <br>
#### 3200 grid stochastic
8.145316666666668
16.25061666666667
15.4032
9.455366666666668


In [27]:
"""batch call simulation"""
# batches = 12
# test_calls = CSV.File("../test/austin-data/austin_test_calls_update1.csv")|> DataFrame 
# #update 1: remove negative indices
# using DelimitedFiles

# for i = 1:12
#     """ simulation parameters """
#     p = DeploymentProblem(
#               hourly_calls,
#               adjacent_nbhd,
#               coverage,
#               namb = namb,
#               train_filter = (hourly_calls[!,:year] .== 2019) .* (hourly_calls[!,:month] .<= 3)
#               )
#     test_calls = CSV.File("../test/austin-data/austin_test_calls_update1.csv")|> DataFrame 
#     #update 1: remove negative indices
#     start_ind = (i-1)*1000+1
#     end_ind = (i)*1000
#     println("running $namb ambuances & $start_ind to $end_ind calls")
#     test_calls_batch = test_calls[start_ind:end_ind,:]

#     # distribution of ambulances across all of the stations
#     x = amb_deployment[model_dict[model_name]][namb]

#     #establishes the dispatch problem
#     problem = DispatchProblem(test_calls_batch, hospitals, stations, p.coverage, x, turnaround=turnaround)

#     #establishes the dispatch model
#     dispatch = ClosestDispatch(p, problem)

#     #@time df, guiArray = simulate_events!(problem, dispatch)
#     df, guiArray = simulate_events!(problem, dispatch)
#     print(mean(df["responsetime"]))
    
#     filename = string(model_name,string(namb),"calls",string(ncalls),"batch",string(i),".csv")
#     open(filename, "w") do filename
#         writedlm(filename,  df["responsetime"])
#     end
#     print(filename)
# end

"batch call simulation"

In [28]:
"""all models simulation"""
# turnaround = Distributions.LogNormal(3.65, 0.3)
# lambda = 0
# local_path = ""
# model_name = "Stochastic"
# namb = 40
# ncalls = 1000
# #using AmbulanceDeployment

# pwd()

# cd(currentpath * "/test/")
# pwd()

# #using AmbulanceDeployment
# using Dates

# #include("..//src//dispatch/closestdispatch.jl")
# include("..//src//problem.jl")
# #include("..//src//evaluate.jl") #if you include, you need to include JuMP
# #include("..//src//simulate.jl")

# hospitals = CSV.File(string(local_path,"../test/austin-data/hospitals.csv")) |> DataFrame
# stations = CSV.File(string(local_path,"../test/austin-data/stations.csv")) |> DataFrame
# hourly_calls = CSV.File("../test/austin-data/Full_WeekdayCalls.csv") |> DataFrame
# adjacent_nbhd = CSV.File("../test/austin-data/adjacent_nbhd.csv") |> DataFrame
# coverage = CSV.read("../test/austin-data/coverage_real.csv", DataFrame, header=false)
# coverage = convert(Array{Bool, 2}, coverage[:, :])
# incidents = CSV.File("../test/austin-data/austin_incidents.csv") |> DataFrame
# solverstats = JLD.load("../src/outputs/austin_team_stats_1_03.jld")
# amb_deployment = solverstats["amb_deployment"]
# model_dict = Dict{String, Symbol}("Stochastic"=>:Stochastic, "Robust01"=>:Robust01, "Robust005"=>:Robust005, "Robust001"=>:Robust001, "Robust0001"=>:Robust0001,
#           "Robust00001"=>:Robust00001, "MEXCLP"=>:MEXCLP, "MALP"=>:MALP)


# p = DeploymentProblem(
#           hourly_calls,
#           adjacent_nbhd,
#           coverage,
#           namb = namb,
#           train_filter = (hourly_calls[!,:year] .== 2019) .* (hourly_calls[!,:month] .<= 3)
#           )
# test_calls = CSV.File("../test/austin-data/austin_test_calls_update1.csv")|> DataFrame 
# #update 1: remove negative indices
# test_calls = test_calls[1:ncalls,:]
# println("running $namb ambuances & $ncalls calls")

# # distribution of ambulances across all of the stations
# x = amb_deployment[model_dict[model_name]][namb]

# #establishes the dispatch problem
# problem = DispatchProblem(test_calls, hospitals, stations, p.coverage, x, turnaround=turnaround)

# #establishes the dispatch model
# dispatch = ClosestDispatch(p, problem)

# Random.seed!(1234)

"all models simulation"

In [29]:
array = filter(x->typeof(x)==gui_event,guiArray)
responded_array = filter(x->x.event_type == "call responded", array)

1000-element Array{Any,1}:
 gui_event("call responded", 1012, 1, 1, 1, -1, 0, DateTime("2021-07-19T12:39:09.102"))
 gui_event("call responded", 220, 31, 2, 31, -1, 0, DateTime("2021-07-19T12:39:09.196"))
 gui_event("call responded", 1687, 36, 3, 36, -1, 0, DateTime("2021-07-19T12:39:09.258"))
 gui_event("call responded", 1536, 33, 4, 33, -1, 0, DateTime("2021-07-19T12:39:09.304"))
 gui_event("call responded", 1621, 14, 5, 14, -1, 0, DateTime("2021-07-19T12:39:09.397"))
 gui_event("call responded", 578, 22, 6, 22, -1, 0, DateTime("2021-07-19T12:39:09.49"))
 gui_event("call responded", 1169, 16, 7, 16, -1, 0, DateTime("2021-07-19T12:39:09.505"))
 gui_event("call responded", 825, 8, 8, 8, -1, 0, DateTime("2021-07-19T12:39:09.721"))
 gui_event("call responded", 641, 9, 9, 9, -1, 0, DateTime("2021-07-19T12:39:09.8"))
 gui_event("call responded", 825, 3, 10, 3, -1, 0, DateTime("2021-07-19T12:39:09.892"))
 gui_event("call responded", 765, 35, 11, 35, -1, 0, DateTime("2021-07-19T12:39:09.985")

In [61]:
using DelimitedFiles
filename = string("visualization/time_adjust/",model_name,string(namb),"calls",string(ncalls),"_3200grid",".csv")
open(filename, "w") do filename
    writedlm(filename,  df[!,"responsetime"])
end
print(filename)

visualization/time_adjust/Stochastic40calls1000_3200grid.csv

In [48]:
#using CSV
#filename = string(joshpath,"test/",model_name,string(namb),"calls",string(ncalls),".csv")
#CSV.write(filename, df, delim=",")